In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [2]:
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding="latin-1")
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df = df.iloc[:,:2]
df.head(5)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5572, 2)

In [5]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [6]:
df = df.rename(columns={'v2':'sms', 'v1':'verdict'})
df.head(5)

,verdict,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df['sms'] = df['sms'].apply(lambda x: x.lower())
df.head(5)

,verdict,sms
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [8]:
em = df['verdict'].unique()
em_num = {}
j=0
for i in em:
    em_num[i] = j;
    j=j+1
df['verdict'] = df['verdict'].map(em_num)
df.head(5)

,verdict,sms
0,0,"go until jurong point, crazy.. available only ..."
1,0,ok lar... joking wif u oni...
2,1,free entry in 2 a wkly comp to win fa cup fina...
3,0,u dun say so early hor... u c already then say...
4,0,"nah i don't think he goes to usf, he lives aro..."


In [9]:
import string
def remove_punc(txt):
     return txt.translate(str.maketrans('','',string.punctuation))
df['sms'] = df['sms'].apply(remove_punc)
df.head(5)

,verdict,sms
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in 2 a wkly comp to win fa cup fina...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...


In [10]:
def remove_num(txt):
    st=""
    for i in txt:
        if not i.isdigit():
            st=st+i
    return st
df['sms'] = df['sms'].apply(remove_num)
df.head()

,verdict,sms
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in a wkly comp to win fa cup final...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...


In [11]:
def remove_emoji(txt):
    st=""
    for i in txt:
        if i.isascii():
            st=st+i
    return st
df['sms'] = df['sms'].apply(remove_emoji)
df.head(5)

,verdict,sms
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in a wkly comp to win fa cup final...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...


In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Before Removing stopwords

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['sms'],df['verdict'],test_size=0.2)

In [14]:
#applying Bag of Words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB

bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)


nb_model = MultinomialNB()
nb_model.fit(X_train_bow, y_train)
pred_bow = nb_model.predict(X_test_bow)

In [15]:
from sklearn.metrics import f1_score
print(f1_score(y_test,pred_bow))


0.9305555555555556


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)

X_test_tfidf = tfidf.transform(X_test)

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

pred_tfidf = nb.predict(X_test_tfidf)


In [17]:
print(f1_score(y_test,pred_tfidf))

0.7836734693877551


# Removing stopwords

In [18]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
nltk.download("punkt")

stop_words = set(stopwords.words('english'))

def remove_sw(txt):
    words = txt.split()
    cleaned = []
    for i in words:
        if i.lower() not in stop_words:
            cleaned.append(i)
    return ' '.join(cleaned)

df['sms'] = df['sms'].apply(remove_sw)
df.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,verdict,sms
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
2,1,free entry wkly comp win fa cup final tkts st ...
3,0,u dun say early hor u c already say
4,0,nah dont think goes usf lives around though


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['sms'],df['verdict'],test_size=0.2,random_state=42)

In [20]:
tf_idf = TfidfVectorizer()

X_train_ti = tf_idf.fit_transform(X_train)
X_test_ti = tf_idf.transform(X_test)

nb = MultinomialNB()
nb.fit(X_train_ti,y_train)
pred_2 = nb.predict(X_test_ti)

In [33]:
print(f1_score(y_test,pred_2))

0.8679245283018869


In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
lrr = LogisticRegression()
lrr.fit(X_train_ti,y_train)
pred_lr = lrr.predict(X_test_ti)

In [30]:
print(f1_score(y_test,pred_lr))

0.7777777777777777


In [25]:
sample_sms = ["I'll call you later, stuck in traffic right now."]
sample_sms_ti = tf_idf.transform(sample_sms)
pred = nb.predict(sample_sms_ti)

if pred==0:
    print("ham")
else:
    print("spam")

ham
